In [138]:
import pandas as pd
import pickle
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [3]:
with open('data/pickles/splitdf21.p', 'rb') as readfile: 
    splitdf21 = pickle.load(readfile)
    
with open('data/pickles/splitdf20.p', 'rb') as readfile: 
    splitdf20 = pickle.load(readfile)
    
with open('data/pickles/splitdf19.p', 'rb') as readfile: 
    splitdf19 = pickle.load(readfile)
    
with open('data/pickles/splitdf18.p', 'rb') as readfile: 
    splitdf18 = pickle.load(readfile)
    
with open('data/pickles/splitdf17.p', 'rb') as readfile: 
    splitdf17 = pickle.load(readfile)

In [32]:
splitdf21

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP,WL,MIN_x,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT_x,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,EFG_PCT_y,FTA_RATE,TM_TOV_PCT_y,OREB_PCT_y,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT
0,22020,1610612744,GSW,Golden State Warriors,0022000001,2020-12-22,GSW @ BKN,L,240,99,37,99,0.374,10,33,0.303,15,23,0.652,13,34,47,26,6,6,18,24,-26.0,86.8,88.4,110.5,111.6,-23.8,-23.2,0.703,1.44,17.0,0.281,0.685,0.466,15.773,0.454,1.0,0.201,113.60,112.0,93.33,112,0.350,0.424,0.232,0.158,0.203,0.538,0.348,0.177,0.315
1,22020,1610612751,BKN,Brooklyn Nets,0022000001,2020-12-22,BKN vs. GSW,W,242,125,42,92,0.457,15,35,0.429,26,32,0.813,13,44,57,24,11,7,20,22,26.0,110.5,111.6,86.8,88.4,23.8,23.2,0.571,1.20,16.0,0.315,0.719,0.534,17.687,0.589,1.0,0.201,113.60,112.0,93.33,112,0.650,0.538,0.348,0.177,0.241,0.424,0.232,0.158,0.281
2,22020,1610612747,LAL,Los Angeles Lakers,0022000002,2020-12-22,LAL vs. LAC,L,240,109,38,81,0.469,9,29,0.310,24,31,0.774,8,37,45,22,4,2,19,20,-7.0,103.2,104.8,109.1,111.5,-5.9,-6.7,0.579,1.16,16.2,0.250,0.731,0.510,17.986,0.576,1.0,0.199,106.00,104.0,86.67,104,0.509,0.525,0.383,0.180,0.182,0.548,0.204,0.150,0.269
3,22020,1610612746,LAC,LA Clippers,0022000002,2020-12-22,LAC @ LAL,W,241,116,44,93,0.473,14,40,0.350,14,19,0.737,11,29,40,22,10,3,16,29,7.0,109.1,111.5,103.2,104.8,5.9,6.7,0.500,1.38,15.8,0.269,0.750,0.490,15.043,0.572,1.0,0.198,106.00,104.0,86.67,104,0.491,0.548,0.204,0.150,0.212,0.525,0.383,0.180,0.250
4,22020,1610612749,MIL,Milwaukee Bucks,0022000003,2020-12-23,MIL @ BOS,L,239,121,46,90,0.511,14,35,0.400,15,18,0.833,11,41,52,19,4,6,16,20,-1.0,117.6,119.8,118.6,122.0,-1.1,-2.2,0.413,1.19,14.3,0.333,0.764,0.570,15.546,0.618,1.0,0.198,102.88,100.5,83.75,101,0.506,0.589,0.200,0.155,0.244,0.564,0.109,0.068,0.236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,22020,1610612759,SAS,San Antonio Spurs,0022001078,2021-05-16,SAS vs. PHX,L,240,121,50,98,0.510,7,24,0.292,14,19,0.737,11,35,46,30,5,5,10,8,-2.0,114.8,117.5,118.5,118.3,-3.7,-0.8,0.600,3.00,20.5,0.255,0.745,0.500,9.491,0.569,1.0,0.198,104.56,103.5,86.25,103,0.479,0.546,0.194,0.095,0.216,0.572,0.038,0.077,0.255
2156,22020,1610612761,TOR,Toronto Raptors,0022001079,2021-05-16,TOR vs. IND,L,240,113,39,97,0.402,12,38,0.316,23,29,0.793,16,32,48,21,8,3,12,12,-12.0,106.8,107.6,118.0,117.9,-11.2,-10.3,0.538,1.75,14.7,0.322,0.729,0.505,11.346,0.515,1.0,0.202,105.84,105.5,87.92,105,0.428,0.464,0.299,0.113,0.271,0.584,0.189,0.123,0.271
2157,22020,1610612754,IND,Indiana Pacers,0022001079,2021-05-16,IND @ TOR,W,239,125,48,95,0.505,15,37,0.405,14,18,0.778,10,37,47,34,8,3,13,19,12.0,118.0,117.9,106.8,107.6,11.2,10.3,0.708,2.62,22.7,0.271,0.678,0.495,12.273,0.607,1.0,0.200,105.84,105.5,87.92,106,0.572,0.584,0.189,0.123,0.208,0.464,0.299,0.113,0.322
2158,22020,1610612764,WAS,Washington Wizards,0022001080,2021-05-16,WAS vs. CHA,W,241,115,44,91,0.484,10,31,0.323,17,21,0.810,10,43,53,22,4,3,14,20,5.0,110.3,112.7,102.3,105.8,8.1,7.0,0.500,1.57,16.1,0.271,0.746,0.533,13.431,0.574,1.0,0.200,105.90,103.0,85.83,102,0.525,0.538,0.231,0.134,0.208,0.468,0.255,0.121,0.254


In [33]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_Home', '_Away'],
                      on=['GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_Home != joined.TEAM_ID_Away]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_Home.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result


In [34]:
keep_columns = ['TEAM_ID', 'TEAM_NAME_x', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'PTS', 'PLUS_MINUS', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'EFG_PCT_y', 'FTA_RATE',
       'TM_TOV_PCT_y', 'OREB_PCT_y', 'OPP_EFG_PCT', 'OPP_FTA_RATE','OPP_TOV_PCT', 'OPP_OREB_PCT', 'PIE', 'OFF_RATING', 
        'DEF_RATING', 'NET_RATING', 'TS_PCT', 'PACE', 'AST_TOV']

In [35]:
model_df21 = splitdf21[keep_columns]
model_df20 = splitdf20[keep_columns]
model_df19 = splitdf19[keep_columns]
model_df18 = splitdf18[keep_columns]
model_df17 = splitdf17[keep_columns]

In [36]:
def note_home_team(df):
    df['HOME_TEAM'] = np.where(df['MATCHUP'].str.contains(' vs. '), 1, 0)

In [37]:
note_home_team(model_df21)
note_home_team(model_df20)
note_home_team(model_df19)
note_home_team(model_df18)
note_home_team(model_df17)

<ipython-input-36-d6736530b5d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HOME_TEAM'] = np.where(df['MATCHUP'].str.contains(' vs. '), 1, 0)


In [38]:
model_df21['PTS_ALLOWED'] = model_df21['PTS'] - model_df21['PLUS_MINUS']
model_df20['PTS_ALLOWED'] = model_df20['PTS'] - model_df20['PLUS_MINUS']
model_df19['PTS_ALLOWED'] = model_df19['PTS'] - model_df19['PLUS_MINUS']
model_df18['PTS_ALLOWED'] = model_df18['PTS'] - model_df18['PLUS_MINUS']
model_df17['PTS_ALLOWED'] = model_df17['PTS'] - model_df17['PLUS_MINUS']

<ipython-input-38-3c8a82f36ac1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df21['PTS_ALLOWED'] = model_df21['PTS'] - model_df21['PLUS_MINUS']
<ipython-input-38-3c8a82f36ac1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df20['PTS_ALLOWED'] = model_df20['PTS'] - model_df20['PLUS_MINUS']
<ipython-input-38-3c8a82f36ac1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [39]:
rol_columns = ['PTS', 'PLUS_MINUS', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'EFG_PCT_y', 'FTA_RATE', 'TM_TOV_PCT_y',
       'OREB_PCT_y', 'OPP_EFG_PCT', 'OPP_FTA_RATE', 'OPP_TOV_PCT',
       'OPP_OREB_PCT', 'PIE', 'OFF_RATING', 'DEF_RATING', 'NET_RATING',
       'TS_PCT', 'PACE', 'AST_TOV', 'PTS_ALLOWED']

In [40]:
def mergelast5(df):
        
    rol_lists5 = {}
    rol_ind5 = {}
    for column in rol_columns:
        rol_lists5[column] = []
        rol_ind5[column] = []
    
    series_dict5 = {}
    for column in rol_columns:
        series_dict5[f'Last5{column}'] = []
        
    for column in rol_columns:
    
        for team in df.groupby('TEAM_ID').groups:
            team_rol = df.groupby('TEAM_ID')[column].rolling(5).mean()[team].shift()
            rol_lists5[column].extend(list(team_rol.values))
            rol_ind5[column].extend(list(team_rol.index))
    
        series_dict5[f'Last5{column}'] = pd.Series(rol_lists5[column], rol_ind5[column], name = f"{column} LAST 5")
    
    for series in series_dict5:
        df = df.merge(series_dict5[series], left_index=True, right_index=True)
        
    return df
    

In [41]:
model_df21 = mergelast5(model_df21)
model_df20 = mergelast5(model_df20)
model_df19 = mergelast5(model_df19)
model_df18 = mergelast5(model_df18)
model_df17 = mergelast5(model_df17)

In [43]:
def mergelast10(df):
    rol_lists10 = {}
    rol_ind10 = {}
    for column in rol_columns:
        rol_lists10[column] = []
        rol_ind10[column] = []

    series_dict10 = {}
    for column in rol_columns:
        series_dict10[f'Last10{column}'] = []
        
    for column in rol_columns:
    
        for team in df.groupby('TEAM_ID').groups:
            team_rol = df.groupby('TEAM_ID')[column].rolling(10).mean()[team].shift()
            rol_lists10[column].extend(list(team_rol.values))
            rol_ind10[column].extend(list(team_rol.index))
    
        series_dict10[f'Last10{column}'] = pd.Series(rol_lists10[column], rol_ind10[column], name = f"{column} LAST 10")
    
    for series in series_dict10:
        df = df.merge(series_dict10[series], left_index=True, right_index=True)
        
    return df
    

In [44]:
model_df21 = mergelast10(model_df21)
model_df20 = mergelast10(model_df20)
model_df19 = mergelast10(model_df19)
model_df18 = mergelast10(model_df18)
model_df17 = mergelast10(model_df17)

In [45]:
def merge_expanding_means(df):
    exp_lists = {}
    exp_ind = {}
    for column in rol_columns:
        exp_lists[column] = []
        exp_ind[column] = []

    exp_series_dict = {}
    for column in rol_columns:
        exp_series_dict[f'Season{column}'] = []
        
    for column in rol_columns:
    
        for team in df.groupby('TEAM_ID').groups:
            team_rol = df.groupby('TEAM_ID')[column].expanding().mean()[team].shift()
            exp_lists[column].extend(list(team_rol.values))
            exp_ind[column].extend(list(team_rol.index))
    
        exp_series_dict[f'Season{column}'] = pd.Series(exp_lists[column], exp_ind[column], name = f"{column} Season")
    
    for series in exp_series_dict:
        df = df.merge(exp_series_dict[series], left_index=True, right_index=True)
        
    return df

In [46]:
model_df21 = merge_expanding_means(model_df21)
model_df20 = merge_expanding_means(model_df20)
model_df19 = merge_expanding_means(model_df19)
model_df18 = merge_expanding_means(model_df18)
model_df17 = merge_expanding_means(model_df17)

In [47]:
def track_win_loss(df):
    df['WIN'] = np.where(df['WL'].str.contains('W'), 1, 0)
    df['LOSS'] = np.where(df['WL'].str.contains('L'), 1, 0)

In [48]:
track_win_loss(model_df21)
track_win_loss(model_df20)
track_win_loss(model_df19)
track_win_loss(model_df18)
track_win_loss(model_df17)

In [49]:
def keep_wins(df):
    wins_total = []
    wins_ind = []
    
    for team in df.groupby('TEAM_ID').groups:
        teamwins = df.groupby('TEAM_ID')['WIN'].expanding().sum()[team].shift()
        wins_total.extend(list(teamwins.values))
        wins_ind.extend(list(teamwins.index))
        
    wins_series = pd.Series(wins_total, wins_ind, name='WINS_SEASON')
    df['WINS_SEASON'] = wins_series
    return (df)

In [50]:
keep_wins(model_df21)
keep_wins(model_df20)
keep_wins(model_df19)
keep_wins(model_df18)
keep_wins(model_df17)

,TEAM_ID,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,PLUS_MINUS,FG_PCT,FG3_PCT,FT_PCT,EFG_PCT_y,FTA_RATE,TM_TOV_PCT_y,OREB_PCT_y,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT,PIE,OFF_RATING,DEF_RATING,NET_RATING,TS_PCT,PACE,AST_TOV,HOME_TEAM,PTS_ALLOWED,PTS LAST 5,PLUS_MINUS LAST 5,FG_PCT LAST 5,FG3_PCT LAST 5,FT_PCT LAST 5,EFG_PCT_y LAST 5,FTA_RATE LAST 5,TM_TOV_PCT_y LAST 5,OREB_PCT_y LAST 5,OPP_EFG_PCT LAST 5,OPP_FTA_RATE LAST 5,OPP_TOV_PCT LAST 5,OPP_OREB_PCT LAST 5,PIE LAST 5,OFF_RATING LAST 5,DEF_RATING LAST 5,NET_RATING LAST 5,TS_PCT LAST 5,PACE LAST 5,AST_TOV LAST 5,PTS_ALLOWED LAST 5,PTS LAST 10,PLUS_MINUS LAST 10,FG_PCT LAST 10,FG3_PCT LAST 10,FT_PCT LAST 10,EFG_PCT_y LAST 10,FTA_RATE LAST 10,TM_TOV_PCT_y LAST 10,OREB_PCT_y LAST 10,OPP_EFG_PCT LAST 10,OPP_FTA_RATE LAST 10,OPP_TOV_PCT LAST 10,OPP_OREB_PCT LAST 10,PIE LAST 10,OFF_RATING LAST 10,DEF_RATING LAST 10,NET_RATING LAST 10,TS_PCT LAST 10,PACE LAST 10,AST_TOV LAST 10,PTS_ALLOWED LAST 10,PTS Season,PLUS_MINUS Season,FG_PCT Season,FG3_PCT Season,FT_PCT Season,EFG_PCT_y Season,FTA_RATE Season,TM_TOV_PCT_y Season,OREB_PCT_y Season,OPP_EFG_PCT Season,OPP_FTA_RATE Season,OPP_TOV_PCT Season,OPP_OREB_PCT Season,PIE Season,OFF_RATING Season,DEF_RATING Season,NET_RATING Season,TS_PCT Season,PACE Season,AST_TOV Season,PTS_ALLOWED Season,WIN,LOSS,WINS_SEASON
0,1610612752,New York Knicks,0021600001,2016-10-25,NYK @ CLE,L,88,-29.0,0.368,0.333,0.750,0.420,0.230,0.179,0.220,0.548,0.202,0.141,0.353,0.297,87.1,115.8,-28.7,0.459,101.0,0.94,0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN
1,1610612739,Cleveland Cavaliers,0021600001,2016-10-25,CLE vs. NYK,W,117,29.0,0.479,0.371,0.737,0.548,0.202,0.141,0.216,0.420,0.230,0.179,0.271,0.703,115.8,87.1,28.7,0.572,101.0,2.07,1,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN
2,1610612757,Portland Trail Blazers,0021600002,2016-10-25,POR vs. UTA,W,113,9.0,0.520,0.684,1.000,0.607,0.293,0.140,0.139,0.537,0.195,0.144,0.262,0.548,122.8,114.3,8.5,0.667,91.5,1.69,1,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN
3,1610612762,Utah Jazz,0021600002,2016-10-25,UTA @ POR,L,104,-9.0,0.488,0.333,1.000,0.537,0.195,0.144,0.143,0.607,0.293,0.140,0.222,0.452,114.3,122.8,-8.5,0.584,91.5,1.36,0,113.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN
4,1610612759,San Antonio Spurs,0021600003,2016-10-25,SAS @ GSW,W,129,29.0,0.480,0.500,0.885,0.541,0.265,0.137,0.396,0.512,0.212,0.159,0.208,0.600,129.0,99.0,30.0,0.589,100.5,1.79,0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,1610612758,Sacramento Kings,0021601228,201

In [51]:
def keep_ls(df):
    loss_total = []
    loss_ind = []
    
    for team in df.groupby('TEAM_ID').groups:
        teamloss = df.groupby('TEAM_ID')['LOSS'].expanding().sum()[team].shift()
        loss_total.extend(list(teamloss.values))
        loss_ind.extend(list(teamloss.index))
        
    loss_series = pd.Series(loss_total, loss_ind, name='LOSS_SEASON')
    df['LOSS_SEASON'] = loss_series
    return (df)

In [52]:
keep_ls(model_df21)
keep_ls(model_df20)
keep_ls(model_df19)
keep_ls(model_df18)
keep_ls(model_df17)

,TEAM_ID,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,PLUS_MINUS,FG_PCT,FG3_PCT,FT_PCT,EFG_PCT_y,FTA_RATE,TM_TOV_PCT_y,OREB_PCT_y,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT,PIE,OFF_RATING,DEF_RATING,NET_RATING,TS_PCT,PACE,AST_TOV,HOME_TEAM,PTS_ALLOWED,PTS LAST 5,PLUS_MINUS LAST 5,FG_PCT LAST 5,FG3_PCT LAST 5,FT_PCT LAST 5,EFG_PCT_y LAST 5,FTA_RATE LAST 5,TM_TOV_PCT_y LAST 5,OREB_PCT_y LAST 5,OPP_EFG_PCT LAST 5,OPP_FTA_RATE LAST 5,OPP_TOV_PCT LAST 5,OPP_OREB_PCT LAST 5,PIE LAST 5,OFF_RATING LAST 5,DEF_RATING LAST 5,NET_RATING LAST 5,TS_PCT LAST 5,PACE LAST 5,AST_TOV LAST 5,PTS_ALLOWED LAST 5,PTS LAST 10,PLUS_MINUS LAST 10,FG_PCT LAST 10,FG3_PCT LAST 10,FT_PCT LAST 10,EFG_PCT_y LAST 10,FTA_RATE LAST 10,TM_TOV_PCT_y LAST 10,OREB_PCT_y LAST 10,OPP_EFG_PCT LAST 10,OPP_FTA_RATE LAST 10,OPP_TOV_PCT LAST 10,OPP_OREB_PCT LAST 10,PIE LAST 10,OFF_RATING LAST 10,DEF_RATING LAST 10,NET_RATING LAST 10,TS_PCT LAST 10,PACE LAST 10,AST_TOV LAST 10,PTS_ALLOWED LAST 10,PTS Season,PLUS_MINUS Season,FG_PCT Season,FG3_PCT Season,FT_PCT Season,EFG_PCT_y Season,FTA_RATE Season,TM_TOV_PCT_y Season,OREB_PCT_y Season,OPP_EFG_PCT Season,OPP_FTA_RATE Season,OPP_TOV_PCT Season,OPP_OREB_PCT Season,PIE Season,OFF_RATING Season,DEF_RATING Season,NET_RATING Season,TS_PCT Season,PACE Season,AST_TOV Season,PTS_ALLOWED Season,WIN,LOSS,WINS_SEASON,LOSS_SEASON
0,1610612752,New York Knicks,0021600001,2016-10-25,NYK @ CLE,L,88,-29.0,0.368,0.333,0.750,0.420,0.230,0.179,0.220,0.548,0.202,0.141,0.353,0.297,87.1,115.8,-28.7,0.459,101.0,0.94,0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN
1,1610612739,Cleveland Cavaliers,0021600001,2016-10-25,CLE vs. NYK,W,117,29.0,0.479,0.371,0.737,0.548,0.202,0.141,0.216,0.420,0.230,0.179,0.271,0.703,115.8,87.1,28.7,0.572,101.0,2.07,1,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN
2,1610612757,Portland Trail Blazers,0021600002,2016-10-25,POR vs. UTA,W,113,9.0,0.520,0.684,1.000,0.607,0.293,0.140,0.139,0.537,0.195,0.144,0.262,0.548,122.8,114.3,8.5,0.667,91.5,1.69,1,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN
3,1610612762,Utah Jazz,0021600002,2016-10-25,UTA @ POR,L,104,-9.0,0.488,0.333,1.000,0.537,0.195,0.144,0.143,0.607,0.293,0.140,0.222,0.452,114.3,122.8,-8.5,0.584,91.5,1.36,0,113.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN
4,1610612759,San Antonio Spurs,0021600003,2016-10-25,SAS @ GSW,W,129,29.0,0.480,0.500,0.885,0.541,0.265,0.137,0.396,0.512,0.212,0.159,0.208,0.600,129.0,99.0,30.0,0.589,100.5,1.79,0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,161061

In [53]:
def win_per(df):
    df['WIN_%'] = (df['WINS_SEASON'] / df['LOSS_SEASON'] + df['WINS_SEASON']) / 100
    

In [54]:
win_per(model_df21)
win_per(model_df20)
win_per(model_df19)
win_per(model_df18)
win_per(model_df17)

In [60]:
import datetime 
import time

In [55]:
def yesterday(date):
    today = datetime.datetime.strptime(date, '%Y-%m-%d')
    yesterday = datetime.datetime.strftime(today - datetime.timedelta(1), '%Y-%m-%d')
    return yesterday

In [56]:
def yesterday_apply(df):
    df['previous_day'] = df.apply(lambda row: yesterday(date=row['GAME_DATE']), axis=1)
    

In [57]:
def played_yesterday(df, date, team_id):
    filtered_df = df.loc[df.GAME_DATE == date]
    filtered_df = filtered_df.loc[filtered_df.TEAM_ID == team_id]
    if filtered_df.empty:
        return 0
    else:
        return 1

In [58]:
def played_yesterday_apply(df):
    df['back_to_back'] = df.apply(lambda row: played_yesterday(df, date=row['previous_day'], team_id = row['TEAM_ID']), axis=1)

In [61]:
yesterday_apply(model_df21)
yesterday_apply(model_df20)
yesterday_apply(model_df19)
yesterday_apply(model_df18)
yesterday_apply(model_df17)

,TEAM_ID,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,PLUS_MINUS,FG_PCT,FG3_PCT,FT_PCT,EFG_PCT_y,FTA_RATE,TM_TOV_PCT_y,OREB_PCT_y,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT,PIE,OFF_RATING,DEF_RATING,NET_RATING,TS_PCT,PACE,AST_TOV,HOME_TEAM,PTS_ALLOWED,PTS LAST 5,PLUS_MINUS LAST 5,FG_PCT LAST 5,FG3_PCT LAST 5,FT_PCT LAST 5,EFG_PCT_y LAST 5,FTA_RATE LAST 5,TM_TOV_PCT_y LAST 5,OREB_PCT_y LAST 5,OPP_EFG_PCT LAST 5,OPP_FTA_RATE LAST 5,OPP_TOV_PCT LAST 5,OPP_OREB_PCT LAST 5,PIE LAST 5,OFF_RATING LAST 5,DEF_RATING LAST 5,NET_RATING LAST 5,TS_PCT LAST 5,PACE LAST 5,AST_TOV LAST 5,PTS_ALLOWED LAST 5,PTS LAST 10,PLUS_MINUS LAST 10,FG_PCT LAST 10,FG3_PCT LAST 10,FT_PCT LAST 10,EFG_PCT_y LAST 10,FTA_RATE LAST 10,TM_TOV_PCT_y LAST 10,OREB_PCT_y LAST 10,OPP_EFG_PCT LAST 10,OPP_FTA_RATE LAST 10,OPP_TOV_PCT LAST 10,OPP_OREB_PCT LAST 10,PIE LAST 10,OFF_RATING LAST 10,DEF_RATING LAST 10,NET_RATING LAST 10,TS_PCT LAST 10,PACE LAST 10,AST_TOV LAST 10,PTS_ALLOWED LAST 10,PTS Season,PLUS_MINUS Season,FG_PCT Season,FG3_PCT Season,FT_PCT Season,EFG_PCT_y Season,FTA_RATE Season,TM_TOV_PCT_y Season,OREB_PCT_y Season,OPP_EFG_PCT Season,OPP_FTA_RATE Season,OPP_TOV_PCT Season,OPP_OREB_PCT Season,PIE Season,OFF_RATING Season,DEF_RATING Season,NET_RATING Season,TS_PCT Season,PACE Season,AST_TOV Season,PTS_ALLOWED Season,WIN,LOSS,WINS_SEASON,LOSS_SEASON,WIN_%,previous_day
0,1610612752,New York Knicks,0021600001,2016-10-25,NYK @ CLE,L,88,-29.0,0.368,0.333,0.750,0.420,0.230,0.179,0.220,0.548,0.202,0.141,0.353,0.297,87.1,115.8,-28.7,0.459,101.0,0.94,0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,2016-10-24
1,1610612739,Cleveland Cavaliers,0021600001,2016-10-25,CLE vs. NYK,W,117,29.0,0.479,0.371,0.737,0.548,0.202,0.141,0.216,0.420,0.230,0.179,0.271,0.703,115.8,87.1,28.7,0.572,101.0,2.07,1,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,2016-10-24
2,1610612757,Portland Trail Blazers,0021600002,2016-10-25,POR vs. UTA,W,113,9.0,0.520,0.684,1.000,0.607,0.293,0.140,0.139,0.537,0.195,0.144,0.262,0.548,122.8,114.3,8.5,0.667,91.5,1.69,1,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,2016-10-24
3,1610612762,Utah Jazz,0021600002,2016-10-25,UTA @ POR,L,104,-9.0,0.488,0.333,1.000,0.537,0.195,0.144,0.143,0.607,0.293,0.140,0.222,0.452,114.3,122.8,-8.5,0.584,91.5,1.36,0,113.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,2016-10-24
4,1610612759,San Antonio Spurs,0021600003,2016-10-25,SAS @ GSW,W,129,29.0,0.480,0.500,0.885,0.541,0.265,0.137,0.396,0.512,0.212,0.159,0.208,0.600,129.0,99.0,30.0,0.589,100.5,1.79,0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,2016-10-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [62]:
played_yesterday_apply(model_df21)
played_yesterday_apply(model_df20)
played_yesterday_apply(model_df19)
played_yesterday_apply(model_df18)
played_yesterday_apply(model_df17)

In [104]:
def merge_home(df):
    exp_lists_home = {}
    exp_ind_home = {}
    for column in rol_columns:
        exp_lists_home[column] = []
        exp_ind_home[column] = []

    home_series_dict = {}
    for column in rol_columns:
        home_series_dict[f'Home{column}'] = []
        
    for column in rol_columns:
    
        for team in df.groupby('TEAM_ID').groups:
            home_df = df.loc[df.HOME_TEAM == 1]
            home_team_rol = home_df.groupby('TEAM_ID')[column].expanding().mean()[team].shift()
            exp_lists_home[column].extend(list(home_team_rol.values))
            exp_ind_home[column].extend(list(home_team_rol.index))
            
            
        home_series_dict[f'Home{column}'] = pd.Series(exp_lists_home[column], exp_ind_home[column], name = f"{column} Home")
        
    for series in home_series_dict:
        home_df = home_df.merge(home_series_dict[series], left_index=True, right_index=True)
                
    return home_df

In [115]:
def merge_away(df):
        
    exp_lists_away = {}
    exp_ind_away = {}
    for column in rol_columns:
        exp_lists_away[column] = []
        exp_ind_away[column] = []

    away_series_dict = {}
    for column in rol_columns:
        away_series_dict[f'Away{column}'] = []
        
    for column in rol_columns:
    
        for team in df.groupby('TEAM_ID').groups:
            away_df = df.loc[df.HOME_TEAM == 0]
            away_team_rol = away_df.groupby('TEAM_ID')[column].expanding().mean()[team].shift()
            exp_lists_away[column].extend(list(away_team_rol.values))
            exp_ind_away[column].extend(list(away_team_rol.index))
                       
    
        away_series_dict[f'Away{column}'] = pd.Series(exp_lists_away[column], exp_ind_away[column], name = f"{column} Away")
            
                  
    for series in away_series_dict:
        away_df = away_df.merge(away_series_dict[series], left_index=True, right_index=True)
                
    return away_df

In [147]:
home21 = merge_home(model_df21)
home20 = merge_home(model_df20)
home19 = merge_home(model_df19)
home18 = merge_home(model_df18)
home17 = merge_home(model_df17)

In [151]:
home_columns = ['PTS Home', 'PLUS_MINUS Home', 'FG_PCT Home', 'FG3_PCT Home',
       'FT_PCT Home', 'EFG_PCT_y Home', 'FTA_RATE Home', 'TM_TOV_PCT_y Home',
       'OREB_PCT_y Home', 'OPP_EFG_PCT Home', 'OPP_FTA_RATE Home',
       'OPP_TOV_PCT Home', 'OPP_OREB_PCT Home', 'PIE Home', 'OFF_RATING Home',
       'DEF_RATING Home', 'NET_RATING Home', 'TS_PCT Home', 'PACE Home',
       'AST_TOV Home', 'PTS_ALLOWED Home']

In [116]:
away21 = merge_away(model_df21)
away20 = merge_away(model_df20)
away19 = merge_away(model_df19)
away18 = merge_away(model_df18)
away17 = merge_away(model_df17)

In [150]:
away_columns = ['PTS Away', 'PLUS_MINUS Away', 'FG_PCT Away', 'FG3_PCT Away',
       'FT_PCT Away', 'EFG_PCT_y Away', 'FTA_RATE Away', 'TM_TOV_PCT_y Away',
       'OREB_PCT_y Away', 'OPP_EFG_PCT Away', 'OPP_FTA_RATE Away',
       'OPP_TOV_PCT Away', 'OPP_OREB_PCT Away', 'PIE Away', 'OFF_RATING Away',
       'DEF_RATING Away', 'NET_RATING Away', 'TS_PCT Away', 'PACE Away',
       'AST_TOV Away', 'PTS_ALLOWED Away']

In [153]:
home21 = home21[home_columns]
home20 = home20[home_columns]
home19 = home19[home_columns]
home18 = home18[home_columns]
home17 = home17[home_columns]

In [154]:
away21 = away21[away_columns]
away20 = away20[away_columns]
away19 = away19[away_columns]
away18 = away18[away_columns]
away17 = away17[away_columns]

In [168]:
pd.concat([home21, away21])
pd.concat([home20, away20])
pd.concat([home19, away19])
pd.concat([home18, away18])
pd.concat([home17, away17])

,PTS Home,PLUS_MINUS Home,FG_PCT Home,FG3_PCT Home,FT_PCT Home,EFG_PCT_y Home,FTA_RATE Home,TM_TOV_PCT_y Home,OREB_PCT_y Home,OPP_EFG_PCT Home,OPP_FTA_RATE Home,OPP_TOV_PCT Home,OPP_OREB_PCT Home,PIE Home,OFF_RATING Home,DEF_RATING Home,NET_RATING Home,TS_PCT Home,PACE Home,AST_TOV Home,PTS_ALLOWED Home,PTS Away,PLUS_MINUS Away,FG_PCT Away,FG3_PCT Away,FT_PCT Away,EFG_PCT_y Away,FTA_RATE Away,TM_TOV_PCT_y Away,OREB_PCT_y Away,OPP_EFG_PCT Away,OPP_FTA_RATE Away,OPP_TOV_PCT Away,OPP_OREB_PCT Away,PIE Away,OFF_RATING Away,DEF_RATING Away,NET_RATING Away,TS_PCT Away,PACE Away,AST_TOV Away,PTS_ALLOWED Away
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.750,-1.400,0.446625,0.338875,0.722625,0.500150,0.285725,0.157900,0.190475,0.498575,0.234025,0.147675,0.270825,0.504675,102.7400,104.0950,-1.3675,0.536475,97.97500,1.54925,102.15
2450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.225,6.475,0.467875,0.384050,0.804025,0.520725,0.267975,0.137875,0.219675,0.491050,0.268075,0.146650,0.274575,0.544625,110.3350,103.3850,6.9450,0.561925,94.71725,1.79800,98.75
2453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.700,-5.550,0.453200,0.370800,0.750550,0.505350,0.279000,0.147650,0.194550,0.528850,0.297475,0.146850,0.282675,0.465400,104.7050,110.7875,-6.0750,0.542850,94.90125,1.62225,106.25
2454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.775,-12.125,0.436500,0.328150,0.762225,0.486025,0.260175,0.154400,0.213950,0.549000,0.289975,0.139775,0.285550,0.414425,102.2075,114.2200,-12.0100,0.524650,99.71250,1.41225,113.90


In [191]:
#model_df21 = model_df21.merge(pd.concat([home21, away21]), left_index=True, right_index=True)
#model_df20 = model_df20.merge(pd.concat([home20, away20]), left_index=True, right_index=True)
# model_df19 = model_df19.merge(pd.concat([home19, away19]), left_index=True, right_index=True)
# model_df18 = model_df18.merge(pd.concat([home18, away18]), left_index=True, right_index=True)
# model_df17 = model_df17.merge(pd.concat([home17, away17]), left_index=True, right_index=True)

In [197]:
with open('data/pickles/model_df21.p', 'wb') as writefile: 
    pickle.dump(model_df21, writefile)
    
with open('data/pickles/model_df20.p', 'wb') as writefile: 
    pickle.dump(model_df20, writefile)
    
with open('data/pickles/model_df19.p', 'wb') as writefile: 
    pickle.dump(model_df19, writefile)
    
with open('data/pickles/model_df18.p', 'wb') as writefile: 
    pickle.dump(model_df18, writefile)
    
with open('data/pickles/model_df17.p', 'wb') as writefile: 
    pickle.dump(model_df17, writefile)